In [1]:
# Download all necessary dependencies
# In this case we need pandas for dealing with Excel files and requests for talking to mdcalc
!pip install -r requirements.txt

In [2]:
import os
import json
import requests
from requests.adapters import HTTPAdapter, Retry
import pandas as pd
from typing import List

In [3]:
# Paste the absolute path to the Excel file here
path_to_file = "/Users/yuvaltimen/Desktop/example data for hypertension.xlsx"  

# Paste the absolute path to the Excel file here
# Creates it if it doesn't already exist
output_file_path = "/Users/yuvaltimen/Desktop/excel_results.xlsx"
assert(os.path.exists(path_to_file))


# About

## Data
Input columns 
- Gender
- age
- height (cm)	
- sysbp	
- diask5

Target output column:
- Category

Each row is a specific instance that blood pressure was taken. 
The website (linked [here](https://www.mdcalc.com/calc/4052/aap-pediatric-hypertension-guidelines)) takes in their age, sex, height, and systolic and diastolic blood pressure and gives me the answer 
"Elevated, Stage 1 HTN, Stage 2 HTN, or Normal BP". 
That's what I need to be put into the column in blue labelled "Categories".

## Expected number of rows 
Could be up to 1000 but usually not more than 100


In [4]:
df = pd.read_excel(path_to_file)
print(f"Found {df.shape[0]} rows and {df.shape[1]} columns")
print("Showing first 5 rows here:")
df.head(5)

Found 9 rows and 17 columns
Showing first 5 rows here:


,MRN,CSN,LOCATION,Appt_Clinic_Name,Sex,Gender,age,Performed by,Category,Method,Event dat e,Event date,Event date,height (cm),sysbp,diask5,Over thirteen?
0,1000000,NaN,NaN,NaN,NaN,M,3,NaN,NaN,NaN,NaN,NaN,NaN,105,94,65,NaN
1,1000001,NaN,NaN,NaN,NaN,F,4,NaN,NaN,NaN,NaN,NaN,NaN,108,98,55,NaN
2,1000002,NaN,NaN,NaN,NaN,M,5,NaN,NaN,NaN,NaN,NaN,NaN,117,97,60,NaN
3,1000003,NaN,NaN,NaN,NaN,F,6,NaN,NaN,NaN,NaN,NaN,NaN,129,95,62,NaN
4,1000004,NaN,NaN,NaN,NaN,M,7,NaN,NaN,NaN,NaN,NaN,NaN,127,96,65,NaN


In [5]:
# Sends requests to https://www.mdcalc.com/calc/4052/aap-pediatric-hypertension-guidelines
# Uses retries since this website is garbage and takes forever to calculate a single response
def compute_bloodpressure_levels(df: pd.DataFrame) -> List[str]:
    """
    is_female: bool, 
    age: float, 
    height_cm: float,
    sysbp: float,

    diask5: float
    """
    url_endpoint = "https://www.mdcalc.com/api/v1/calc/4052/calculate"
    results = []
    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[])
    s.mount("http://", HTTPAdapter(max_retries=retries))

    for row in df.iterrows():
        row = row[1]
        
        request_payload = {
            "UOMSYSTEM":False,  # False for cm, True for inches
            "age": row["age"],
            "sex": 1 if row["Gender"] == "M" else 0,
            "ht": row["height (cm)"],
            "sbp": row["sysbp"],
            "dbp": row["diask5"],
            "webLanguage":"english"
        }

        response = s.post(url_endpoint, json=request_payload)
        if response.ok:
            results.append(json.loads(response.text)['output'][0]['value'])
        else:
            print(response.status_code)
            response.raise_for_status()

    return results

In [6]:
results = compute_bloodpressure_levels(df)
print(results)


['Elevated BP', 'Normal BP', 'Normal BP', 'Normal BP', 'Normal BP', 'Normal BP', 'Stage 1 HTN', 'Elevated BP', 'Stage 2 HTN']


In [7]:
# Set the 'Category' column to the results and save the Excel file
df['Category'] = results
df.head(5)

,MRN,CSN,LOCATION,Appt_Clinic_Name,Sex,Gender,age,Performed by,Category,Method,Event dat e,Event date,Event date,height (cm),sysbp,diask5,Over thirteen?
0,1000000,NaN,NaN,NaN,NaN,M,3,NaN,Elevated BP,NaN,NaN,NaN,NaN,105,94,65,NaN
1,1000001,NaN,NaN,NaN,NaN,F,4,NaN,Normal BP,NaN,NaN,NaN,NaN,108,98,55,NaN
2,1000002,NaN,NaN,NaN,NaN,M,5,NaN,Normal BP,NaN,NaN,NaN,NaN,117,97,60,NaN
3,1000003,NaN,NaN,NaN,NaN,F,6,NaN,Normal BP,NaN,NaN,NaN,NaN,129,95,62,NaN
4,1000004,NaN,NaN,NaN,NaN,M,7,NaN,Normal BP,NaN,NaN,NaN,NaN,127,96,65,NaN


In [8]:
print(output_file_path)
if not os.path.exists(output_file_path):
    open(output_file_path, "a").close()
    
df.to_excel(output_file_path)

/Users/yuvaltimen/Desktop/excel_results.xlsx


In [9]:
print("All done!")

All done!
